In [5]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

from data_split import data_split_generator
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import numpy as np
from tqdm import tqdm
from itertools import product

In [17]:
[x for x in product(range(10), range(3))]

[(0, 0),
 (0, 1),
 (0, 2),
 (1, 0),
 (1, 1),
 (1, 2),
 (2, 0),
 (2, 1),
 (2, 2),
 (3, 0),
 (3, 1),
 (3, 2),
 (4, 0),
 (4, 1),
 (4, 2),
 (5, 0),
 (5, 1),
 (5, 2),
 (6, 0),
 (6, 1),
 (6, 2),
 (7, 0),
 (7, 1),
 (7, 2),
 (8, 0),
 (8, 1),
 (8, 2),
 (9, 0),
 (9, 1),
 (9, 2)]

In [21]:
def make_x_y(data: pd.DataFrame):
    data = data.copy(deep=True)
    data.reset_index(drop=True, inplace=True)
    # data['']
    data.drop(columns=['dates', 'target_dates'], inplace=True)
    # month = month_encoder.transform(data[['month']]).toarray()
    # month = pd.DataFrame(month, columns=[str(x) for x in month_encoder.categories_[0].tolist()])
    # data:pd.DataFrame = pd.concat([data, month], axis = 1)
    Y = data.pop('target_sunspots')
    return data , Y
def get_metric(y_true, y_predict):
    RMSE = np.sqrt(mean_squared_error(y_true, y_predict))
    # MAE = mean_absolute_error(y_true, y_predict)
    # MAPE = mean_absolute_percentage_error(y_true, y_predict)
    return RMSE

# train_x, train_y, month_encoder = make_x_y(train_data)
# train_x

In [27]:
final_metric = []
for j in tqdm(range(120, 133)):
    data = pd.read_csv(f'./data/pca_data_{j}.csv')
    data['target_dates'] = pd.to_datetime(data['target_dates'])
    n_estimators = range(40, 51, 5)
    max_depth = range(40, 51, 5)
    criterion = ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']
    for n_estimators, max_depth, criterion in product(n_estimators, max_depth, criterion):
        metrics = []
        data_split = data_split_generator(data)
        for train_data, vali_data in data_split:
            train_x, train_y = make_x_y(train_data)
            rf = RandomForestRegressor(n_jobs=6, n_estimators=n_estimators, max_depth=max_depth, criterion=criterion)
            rf.fit(X=train_x, y=train_y)
            vali_x, vali_y = make_x_y(vali_data)
            predict = rf.predict(vali_x)
            metric = get_metric(vali_y, predict)
            metrics.append(metric)
        metrics = np.mean(metrics)
        final_metric.append([j, n_estimators, max_depth, criterion, metrics])

final_metric = pd.DataFrame(final_metric, columns=['m', 'n_estimators', 'max_depth', 'criterion', 'metrics'])
final_metric

100%|██████████| 13/13 [02:02<00:00,  9.43s/it]


,m,n_estimators,max_depth,criterion,metrics
0,120,40,40,squared_error,41.250023
1,120,40,40,absolute_error,40.460128
2,120,40,40,friedman_mse,40.449779
3,120,40,40,poisson,40.537333
4,120,40,45,squared_error,41.535371
...,...,...,...,...,...
463,132,50,45,poisson,39.844058
464,132,50,50,squared_error,41.208686
465,132,50,50,absolute_error,41.456766
466,132,50,50,friedman_mse,41.169469


In [28]:
final_metric[final_metric.m == 120].groupby('max_depth').mean()

/var/folders/ph/g14_8mqx0kb1gfr_kzfhtfvr0000gn/T/ipykernel_30735/3729959501.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  final_metric[final_metric.m == 120].groupby('max_depth').mean()


,m,n_estimators,metrics
max_depth,,,
40,120.0,45.0,40.713972
45,120.0,45.0,40.851335
50,120.0,45.0,40.755335


In [30]:
final_metric["rank"] = final_metric.groupby(['m'])['metrics'].rank(method="dense", ascending=True)
final_metric[final_metric['rank'] == 1.0]

,m,n_estimators,max_depth,criterion,metrics,rank
29,120,50,45,absolute_error,40.011518,1.0
63,121,50,40,poisson,39.719431,1.0
84,122,45,40,squared_error,39.882804,1.0
109,123,40,40,absolute_error,42.479966,1.0
146,124,40,40,friedman_mse,41.172862,1.0
201,125,45,50,absolute_error,40.441631,1.0
233,126,45,45,absolute_error,40.004152,1.0
267,127,45,40,poisson,39.050921,1.0
301,128,45,40,absolute_error,38.571412,1.0
327,129,40,40,poisson,38.792511,1.0
